In [ ]:
# default_exp simulation

In [ ]:
#hide
#For running in JupyterHub:
import os
if os.path.basename(os.getcwd())!='nbs':
    homedir = os.path.expanduser("~")
    lib_path = os.path.join(homedir,'images','codesDIR/vertex_simulation/nbs')
    os.chdir(lib_path)

# Simulation

> Tools for  simulating cellular vertex dynamics. Iterative algorithm implementations, cell monolayer generators (vertices). (anything else to add?!)

- numerical integration method(s)
    - Euler's method
    - any other approximation?
- implement functions for checking:
    - vertex configuration (for high tension cases there might be overlapping, or twisted cells which are considered illigal by the model)
    - update steps that implement required vertex check, and any other checks needed before accepting vertex position update
- implement Voronoi cell generator for different cell configuration cases (random, evenly spaced, concave cell, ...)

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
from vertex_simulation.primitives import *
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import torch, numpy as np, matplotlib.pyplot as plt
from scipy.spatial import Voronoi,voronoi_plot_2d

## Vertex Trajectory Simulations

In [ ]:
#export
class Simulation(object):
    '''Abstract class for vertex dynamics simulations
    
    Children should implement following methods:
    - `_energy(self,...)`: calculates systems energy
    - `_force(self,...)`: calculates forces acting on all vertices
    (you can implement either one, or both of the energy and force functions)
    - `sample_trajectory(self,...)`: simulates dynamics and samples vertex trajectories.
    This method should use either force or energy (spatial gradient) to simulate the system dynamics.
    '''
    def __init__(self,m=None):
        '''m: `Monolayer` or `Graph` object (e.g. cells defined as polygons).'''
        pass
    def _energy(self):
        '''Computes the total free energy of the system.'''
        pass
    def _force(self):
        '''Computes the total force acting on each vertex. Alternatively use F_i = -grad_i(U), where
        grad_i is the spatial gradient with respect to vertex i, and U is the total free energy.
        '''
        pass
    def sample_trajectory(self, T=10000, delta_T=0.001, sample_freq=10, T_ignore=500):
        '''
        Run simulation for `T` time steps and sample vertex trajectories with frequency `sample_freq`.
        - `T`  : total number of time steps
        - `delta_T`: step size (e.g. for the numerical integration using Euler's method)
        - `sample_freq`: Trajectory sampling frequency. Use `assert (T % sample_freq == 0)` in your implementation.
        - `T_ignore`: number of initial time steps to ignore.
        '''
        pass

In [ ]:
#export
class Simulation_Honda(Simulation):
    '''Honda et al. definition of Vertex model.
    
    Total free energy of the `Monolayer` is defined as `U = Ud + Us + Ua`
    - `Ud` : deformation energy or elastic energy of the cells. For a cell `k`, `Ud[k] = K*(A[k] - Ao)^2`
             where `A[k]>=0` is an area of cell `k`, `K>=0` is an elastic constant, and `Ao>=0` is a target area.
    - `Us` : membrane surface tension energy. For cell `k`, `Us[k] = Kp*(P[k] - Po)^2`
             where `Kp>=0` is constant, `Po` is a target perimeter (constant), and `P[k]` is a perimeter of cell `k`.
    - `Ua` : cell-cell adhesion energy.
    
    References:
    - Fletcher A.G., _et al._, _Progress in Biophysics and Molecular Biology_ __113__, 299-326 (2013).
    '''
    def __init__(self, m=None):
        '''m: `Monolayer` object (`Graph` w/ cells defined as polygons).'''
        super().__init__(m=m)
        
    def _energy(self):
        '''Computes total free energy U. U = Ud + Us + Ua
        '''
        pass

In [ ]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
print(f'Device is set to [ {device} ]')

Device is set to [ cuda ]


In [ ]:
x= torch.zeros(5,5).to(device)
print('x :',x)

a=torch.tensor([0 , .1, -.01, 2, 3]).to(device)
print('a :',a)

x[0,:] = a[:]
print('\n( x[0,:] = a[:] )\nx :',x)

a[0]=-1
print('\n( a[0]=-1 )\na :',a)
print('x :',x)


x : tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], device='cuda:0')
a : tensor([ 0.0000,  0.1000, -0.0100,  2.0000,  3.0000], device='cuda:0')

( x[0,:] = a[:] )
x : tensor([[ 0.0000,  0.1000, -0.0100,  2.0000,  3.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]], device='cuda:0')

( a[0]=-1 )
a : tensor([-1.0000,  0.1000, -0.0100,  2.0000,  3.0000], device='cuda:0')
x : tensor([[ 0.0000,  0.1000, -0.0100,  2.0000,  3.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]], device='cuda:0')


In [ ]:
#hide
# run this as a last cell in your notebook to export this module
from nbdev.export import *
notebook2script()